In [1]:
from NER_cadec import training_loop
import torch

model = training_loop("./datasets/cadec", pos_weight=20, epochs=3, verbose=True)
torch.save(model.state_dict(), "./models/cadec/model_20_3.pth")

/Users/simonelusetti/phd/NER/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Batch loss 2.062427282333374
Batch loss 1.2564388513565063
Batch loss 0.8411583304405212
Batch loss 0.5257542133331299
Batch loss 0.3905346989631653
Batch loss 0.32734444737434387
Batch loss 0.26849621534347534
Batch loss 0.25956711173057556
Batch loss 0.31483903527259827
Batch loss 0.2300548255443573
Batch loss 0.2854592204093933
Batch loss 0.3234803378582001
Batch loss 0.2834727168083191
Batch loss 0.29170528054237366
Batch loss 0.29983803629875183
Batch loss 0.3173629939556122
Batch loss 0.2670811712741852
Batch loss 0.2601722180843353
Batch loss 0.28457918763160706
Batch loss 0.22202657163143158
Epoch 1, Loss: 0.4655896373093128
Batch loss 0.27400001883506775
Batch loss 0.26715707778930664
Batch loss 0.23720552027225494
Batch loss 0.24409396946430206
Batch loss 0.19721704721450806
Batch loss 0.24021771550178528
Batch loss 0.18928980827331543
Batch loss 0.19955332577228546
Batch loss 0.2585411071777344
Batch loss 0.26665833592414856
Batch loss 0.2109128087759018
Batch loss 0.2032873

In [ ]:
from loop_loss_repair import training_loop_repair
from NER_cadec import EntityMatrixPredictor
import torch

model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_20_3.pth"))
model = training_loop("./datasets/cadec", model=model, pos_weight=48, epochs=3, verbose=True)
torch.save(model.state_dict(), "./models/cadec/model_15_6.pth")
model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_15_3.pth"))
model = training_loop_repair("./datasets/cadec", model=model, verbose=True, pos_weight=17, epochs=3, repair_loss_weight=1)
torch.save(model.state_dict(), "./models/cadec/model_repair_1.pth")

Batch loss 0.18455521762371063
Batch loss 0.18826980888843536
Batch loss 0.20899246633052826
Batch loss 0.15383380651474
Batch loss 0.17053207755088806
Batch loss 0.1523893177509308
Batch loss 0.1769653856754303
Batch loss 0.15796466171741486
Batch loss 0.1280408799648285
Batch loss 0.16899144649505615
Batch loss 0.1544807404279709
Batch loss 0.14714688062667847
Batch loss 0.15071214735507965
Batch loss 0.1400219202041626
Batch loss 0.1555113047361374
Batch loss 0.14277061820030212
Batch loss 0.1335197240114212
Batch loss 0.15936806797981262
Batch loss 0.14995788037776947
Batch loss 0.12534719705581665
Epoch 1, Loss: 0.1574685774743557
Batch loss 0.11098252236843109
Batch loss 0.10710667073726654
Batch loss 0.12938857078552246
Batch loss 0.15596279501914978
Batch loss 0.1099814772605896
Batch loss 0.13561192154884338
Batch loss 0.12550851702690125
Batch loss 0.12158408761024475
Batch loss 0.10030084103345871
Batch loss 0.1276055872440338
Batch loss 0.11300407350063324
Batch loss 0.1148

In [5]:
import torch
torch.save(model.state_dict(), "./models/cadec/model_repair_1.pth")

In [2]:
from NER_cadec import evaluation_loop
from NER_cadec import EntityMatrixPredictor
import torch

model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_20_3.pth"))
print(evaluation_loop("./datasets/cadec", model))
"""model = EntityMatrixPredictor(bert_model_name='bert-base-cased')
model.load_state_dict(torch.load("./models/cadec/model_repair_1.pth"))
evaluation_loop("./datasets/cadec", model)"""

(0.11304347826086956, 0.16802721088435374, 0.13515731874145007)


'model = EntityMatrixPredictor(bert_model_name=\'bert-base-cased\')\nmodel.load_state_dict(torch.load("./models/cadec/model_repair_1.pth"))\nevaluation_loop("./datasets/cadec", model)'

In [20]:
from datasets import load_dataset

dataset = load_dataset("ade_corpus_v2")  # Or a specific community version if available
dataset["train"][1:100]

ValueError: Config name is missing.
Please pick one among the available configs: ['Ade_corpus_v2_classification', 'Ade_corpus_v2_drug_ade_relation', 'Ade_corpus_v2_drug_dosage_relation']
Example of usage:
	`load_dataset('ade_corpus_v2', 'Ade_corpus_v2_classification')`

In [9]:
def find_opening_paths(matrix, threshold=0.5, max_distance=10):
    """
    Recursively find all maximal opening paths (as sequences of node indices) in the upper triangle of the matrix.

    Args:
        matrix (torch.Tensor): A square matrix of shape (N, N)
        threshold (float): Minimum value to consider an arc active

    Returns:
        List[List[int]]: Only maximal (non-nested) paths represented as lists of node indices.
    """
    n = matrix.size(0)
    all_paths = []

    def recurse(path, distance=0):
        i,j = path[-1]
        branches = []
        for k in range(i + 1, min(i + distance,n)):
            if matrix[j, k] > threshold: 
                for m in range(k, min(i + distance,n)):
                    if matrix[j, m] > threshold: 
                        branches.append((j, m))
                        matrix[j, m] = 0
                break
        if branches == []: 
            all_paths.append(path)
            return
        for j,m in branches:
            recurse(path + [(j,m)], distance=distance-m)

    for i in range(n):
        for j in range(i + 1, min(i + max_distance, n)):
            if matrix[i, j] > threshold:
                recurse([(i, j)], distance=max_distance)

    return all_paths


In [15]:
def ordered_partitions(lst):
    if not lst:
        yield []
        return
    for i in range(1, len(lst) + 1):
        first = lst[:i]
        for rest in ordered_partitions(lst[i:]):
            yield [first] + rest

In [14]:
def opening_paths_partitions(path):
    """
    Given a forward path as list of arcs (i, j), generate all valid backward-closing partitions.
    Each partition is a list of arcs like (to, from) that would close the entity.

    Args:
        path: List of tuples representing the forward arcs (e.g., [(0,1), (1,2), (2,3)])

    Returns:
        List[List[Tuple[int, int]]]: All possible backward arcs to close the path
    """
    repairs = []
    for p in ordered_partitions(path):
        rep = []
        for e in p:
            rep.append((e[-1][1], e[0][0]))
        repairs.append(rep)

    return repairs

In [13]:
def repair_matrix_lower(logits, threshold=0.5, max_distance=10):
    upper = torch.triu(logits, diagonal=1)
    paths = find_opening_paths(logits, threshold, max_distance)
    for path in paths:
        partitions = opening_paths_partitions(path)
        partitions_weight = []
        for p in partitions:
            partitions_weight.append(sum(list(map(lambda cell: 1-logits[cell[0],cell[1]], p))))

        for repaired_cell in partitions[partitions_weight.index(min(partitions_weight))]:
            logits[repaired_cell[0], repaired_cell[1]] = 1
    return torch.tril(logits, diagonal=-1) + upper

In [16]:
import torch

logits = torch.tensor([
    [0.0, 0.0, 0.0, 0.0, 0.9],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.9],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
])

repair_matrix_lower(logits, threshold=0.5, max_distance=3)  

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.9000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.9000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000]])

In [27]:
def find_closing_paths(logits, threshold=0.5, max_distance=10):
    """
    Recursively find all maximal closing paths (as sequences of node indices) in the lower triangle of the matrix.

    Args:
        matrix (torch.Tensor): A square matrix of shape (N, N)
        threshold (float): Minimum value to consider an arc active

    Returns:
        List[List[int]]: Only maximal (non-nested) paths represented as lists of node indices.
    """
    n = logits.size(0)
    return [(j, i) for i in range(n) for j in range(i + 1, min(i + max_distance, n)) if logits[j, i] > threshold]


In [18]:
def closing_path_partitions(path):
    i,j = path
    partitions = []
    for p in ordered_partitions(list(range(i - j))):
        k,l = j, j
        part = []
        for e in p:
            k = k + len(e)
            part.append((l,k))
            l = l + len(e)
        partitions.append(part)
    return partitions

In [ ]:
def repair_matrix_upper(logits, threshold=0.5, max_distance=10):
    lower = torch.tril(logits, diagonal=-1)
    paths = find_closing_paths(logits, threshold, max_distance)
    for path in paths:
        partitions = closing_path_partitions(path)
        partitions_weight = []
        for p in partitions:
            partitions_weight.append(sum(list(map(lambda cell: 1-logits[cell[0],cell[1]], p))))
        for repaired_cell in partitions[partitions_weight.index(min(partitions_weight))]:
            logits[repaired_cell[0], repaired_cell[1]] = 1
    return lower + torch.triu(logits, diagonal=1)

In [ ]:
logits = torch.tensor([
    [0.0, 0.2, 0.3, 0.4],
    [0.0, 0.0, 0.2, 0.0],
    [0.0, 0.0, 0.0, 0.2],
    [0.9, 0.0, 0.0, 0.0],
])

repair_matrix_upper(logits, threshold=0.5, max_distance=10)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0]])